# This code is to use the Get_initial_cap to Get initial capacity to be Niall's 4.86Ah

In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing
import scipy.optimize

In [2]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2_Crack_simp import (
    GetScan,
    recursive_scan,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,
    Get_initial_cap,
    Get_initial_cap2
)

In [3]:
########################  Input  ########################
""" 12 
[4.8728428390998655, 4.858536253367931] 
[0.8519999999999996, 0.8471043791226032] 
[0.2700000000000001, 0.2710723194914279]"""

# all values here must be a list, even it is a single object
Para_dict_All = {
   "Total ageing cycles":[1170,],
   "Ageing cycles between RPT":[78,],
   "Update cycles for ageing":[26,],
   "Cycles within RPT":[3,],
   "Ageing temperature":[40,],
   "RPT temperature":[25,],
   "Particle mesh points":[30,],   # Simon uses 30
   #"Exponential mesh stretch":[1.0,],
   "Para_Set":[ "OKane2022",],
   "electrolyte":["lipf6_Nyman2008"],
   "Model option":[
         {
            "calculate discharge energy":"true",
            "particle": "Fickian diffusion",          
            "SEI":"solvent-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   "Initial Neg SOC":[0.8471043791226032,],    #list(np.linspace(0.84,0.90,6)),
   "Initial Pos SOC":[0.2710723194914279], # list(np.linspace(0.22,0.27,6)),
   # Solvent consumption sub-model
   "Initial electrolyte excessive amount ratio":[1.05,], # set to <1 for DryOut=Off 
   "Current solvent concentration in the reservoir [mol.m-3]":[4541.0,],
   "Current electrolyte concentration in the reservoir [mol.m-3]":[1000,],
   "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":[1.0,],
   # DFN parameter
   "Upper voltage cut-off [V]":[4.21,],
   "Lower voltage cut-off [V]":[2.49,],
   #"Negative electrode diffusivity [m2.s-1]":[2e-13,], # default: a function, about 3.3e-14
   #"Positive electrode diffusivity [m2.s-1]":[1e-13,], # default: 4e-15
   
   # general SEI
   'SEI resistivity [Ohm.m]':[2E5,],
   # solvent-diffusion limited
   'Outer SEI solvent diffusivity [m2.s-1]':[2.5e-22],       # default: 2.5e-22
   'Bulk solvent concentration [mol.m-3]':[4541.0,],          # default: 4541.0
   # interstitial-diffusion limited
   'Inner SEI lithium interstitial diffusivity [m2.s-1]':[1e-12,],    
   'Lithium interstitial reference concentration [mol.m-3]':[15,],
   # ec-reaction limited
   'EC diffusivity [m2.s-1]':[2e-20,],
   'SEI kinetic rate constant [m.s-1]':[1e-19,], 
   'EC initial concentration in electrolyte [mol.m-3]':[4541.0,],
   # LiP and coupling with SEI:
   "Dead lithium decay constant [s-1]":[ 1e-6,],            # default: 1e-6
   'Lithium plating kinetic rate constant [m.s-1]':[1E-9,], # default: 1e-9
   # Crack model
   "Negative electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Positive electrode LAM constant proportional term [s-1]":[ 2.7778e-9,], # default: 2.7778e-07
   "Negative electrode volume change":[ 0.0,],
   "Positive electrode volume change":[ 0.0,],
}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})


In [4]:
print(Para_dict_list)

[{'Total ageing cycles': 1170, 'Ageing cycles between RPT': 78, 'Update cycles for ageing': 26, 'Cycles within RPT': 3, 'Ageing temperature': 40, 'RPT temperature': 25, 'Particle mesh points': 30, 'Para_Set': 'OKane2022', 'electrolyte': 'lipf6_Nyman2008', 'Model option': {'calculate discharge energy': 'true', 'particle': 'Fickian diffusion', 'SEI': 'solvent-diffusion limited', 'SEI on cracks': 'true', 'SEI film resistance': 'distributed', 'SEI porosity change': 'true', 'particle mechanics': ('swelling and cracking', 'swelling only'), 'loss of active material': 'stress-driven', 'lithium plating': 'partially reversible'}, 'Initial Neg SOC': 0.8471043791226032, 'Initial Pos SOC': 0.2710723194914279, 'Initial electrolyte excessive amount ratio': 1.05, 'Current solvent concentration in the reservoir [mol.m-3]': 4541.0, 'Current electrolyte concentration in the reservoir [mol.m-3]': 1000, 'Ratio of Li-ion concentration change in electrolyte consider solvent consumption': 1.0, 'Upper voltage 

In [5]:
# scan:
index_xlsx = 0; # DryOut option merge to Initial electrolyte excessive amount ratio: On and Off
Cap_all = [];  Neg1SOC_all=[]; Pos1SOC_all= []; Index_all = []
for Para_dict_i in Para_dict_list:
   index_xlsx += 1;
   try:
      Cap, Neg1SOC, Pos1SOC = Get_initial_cap2(Para_dict_i, )
   except:
      pass
   else:
      Cap_all .append(Cap);
      Neg1SOC_all.append(Neg1SOC)
      Pos1SOC_all.append(Pos1SOC)
      Index_all.append(index_xlsx)


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian faile

In [6]:
print(Index_all)

[]


In [7]:
for Index,Cap,Neg1SOC,Pos1SOC in zip(Index_all,Cap_all,Neg1SOC_all,Pos1SOC_all):
      print(
            Index,Cap,
            Neg1SOC,Pos1SOC,)

In [8]:
def Fit_cap ( SOC ):
    Neg1SOC_i, Pos1SOC_i   = SOC
    print('Currently initial SOC is', Neg1SOC_i, " for Neg and ", Pos1SOC_i , " for Pos")
    try:
        Sol_0, Cap, Neg1SOC, Pos1SOC = Get_initial_cap(
            Para_dict_list[0], Neg1SOC_i, Pos1SOC_i
        )
        print(type(Cap))
        print(Cap)
        cap_0 = Cap[-1];
        error = (cap_0-4.86)*(cap_0-4.86)
    except:
        cap_0 = 0;
        error = 1
    else:
        pass
    return error

In [9]:
NegSOC_bounds = (0.8,  0.99);
PosSOC_bounds = (0.15, 0.40);
Neg1SOC = np.random.uniform(
    low=NegSOC_bounds[0], 
    high=NegSOC_bounds[1])
Pos1SOC = np.random.uniform(
    low=PosSOC_bounds[0], 
    high=PosSOC_bounds[1])

print('starting SOC is', Neg1SOC, " for Neg and ", Pos1SOC , " for Pos")
res = scipy.optimize.minimize(
    Fit_cap, [Neg1SOC,Pos1SOC], 
    bounds=[NegSOC_bounds,PosSOC_bounds],
    tol = 1e-2,
    #, jac=True
)


starting SOC is 0.8247802586006052  for Neg and  0.21737831934937263  for Pos
Currently initial SOC is 0.8247802586006052  for Neg and  0.21737831934937263  for Pos


2022-10-14 15:00:59.995 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Events ['Maximum voltage'] are non-positive at initial conditions


Currently initial SOC is 0.8247802686006053  for Neg and  0.21737831934937263  for Pos


2022-10-14 15:01:03.325 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Events ['Maximum voltage'] are non-positive at initial conditions


Currently initial SOC is 0.8247802586006052  for Neg and  0.21737832934937262  for Pos


2022-10-14 15:01:06.673 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Events ['Maximum voltage'] are non-positive at initial conditions


In [10]:
Cap, Neg1SOC, Pos1SOC = Get_initial_cap2(
            Para_dict_list[0]
        )
""" 12 [4.8728428390998655, 4.858536253367931] 
[0.8519999999999996, 0.8471043791226032] 
[0.2700000000000001, 0.2710723194914279] """

psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian faile

SolverError: Maximum number of decreased steps occurred at t=0.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=56.81250000000035) and/or reducing the size of the time steps or period of the experiment.

In [ ]:
print(Cap, Neg1SOC, Pos1SOC)